#### Preprocessing:

In [32]:
import numpy as np
import pandas as pd
import logging
import os
from dotenv import find_dotenv, load_dotenv
import datetime
import glob
from os.path import abspath
from pathlib import Path
from inspect import getsourcefile
from datetime import datetime
import math
import argparse
import sys
import tensorflow as tf

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
import get_raw_data as grd
import data_classes
import Normalizer

DT_FLOAT = np.float32 
DT_BOOL = np.uint8
RANDOM_SEED = 123
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
# logger.propagate = False # it will not log to console.

RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'raw') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')

print(RAW_DIR, PRO_DIR)

C:\Users\sandr\Documents\GitHub\MLMortgage\data\raw C:\Users\sandr\Documents\GitHub\MLMortgage\data\processed


In [17]:
def update_parser(parser):
    """Parse the arguments from the CLI and update the parser."""    
    parser.add_argument(
        '--prepro_step',
        type=str,
        default='preprocessing', #'slicing', 'preprocessing'
        help='To execute a preprocessing method')    
    #this is for allfeatures_preprocessing:
    parser.add_argument(
        '--train_period',
        type=int,
        nargs='*',
        default=[121,323], #[121,279], #[156, 180], [121,143],  # 279],
        help='Training Period')
    parser.add_argument(
        '--valid_period',
        type=int,
        nargs='*',
        default=[324,329], #[280,285], #[181,185], [144,147],
        help='Validation Period')    
    parser.add_argument(
        '--test_period',
        type=int,
        nargs='*',
        default=[330, 342], #[286, 304], # [186,191], [148, 155],
        help='Testing Period')    
    parser.add_argument(
        '--prepro_dir',
        type=str,
        default='chuncks_random_c1mill',
        help='Directory with raw data inside data/raw/ and it will be the output directory inside data/processed/')    
    parser.add_argument(
        '--prepro_chunksize',
        type=int,
        default=500000,
        help='Chunk size to put into the h5 file...')    
    parser.add_argument(
        '--prepro_with_index',
        type=bool,
        default=True,
        help='To keep indexes for each record')
    parser.add_argument(
        '--ref_norm',
        type=bool,
        default=True,
        help='To execute the normalization over the raw inputs')
        
    return parser.parse_known_args()

In [18]:

FLAGS, UNPARSED = update_parser(argparse.ArgumentParser())    
#these are the more important parameters for preprocessing:
FLAGS.prepro_dir='chuncks_random_c1mill' #this directory must be the same inside 'raw' and processed directories.
FLAGS.prepro_chunksize=500000 
FLAGS.train_period=[121,323] #[121,279] #[121, 143] 
FLAGS.valid_period=[324,329] #[280,285] #[144, 147] 
FLAGS.test_period=[330,342] #[286,304] #[148, 155]                                                
FLAGS.prepro_with_index = True

print(FLAGS)    

Namespace(prepro_chunksize=500000, prepro_dir='chuncks_random_c1mill', prepro_step='preprocessing', prepro_with_index=True, ref_norm=True, test_period=[330, 342], train_period=[121, 323], valid_period=[324, 329])


In [19]:
glob.glob(os.path.join(RAW_DIR, FLAGS.prepro_dir,"*.txt"))
# from IPython.core.debugger import Tracer; Tracer()()

['C:\\Users\\sandr\\Documents\\GitHub\\MLMortgage\\data\\raw\\chuncks_random_c1mill\\temporalloandynmodifmrstaticitur15mill-16mill.txt']

In [20]:
def allfeatures_extract_labels(data, columns='MBA_DELINQUENCY_STATUS_next'):
    '''Extract the labels from Dataset, order-and-transform them into one-hot matrix of labels.
        Args: 
            data (DataFrame): Input Dataset which is modified in place.
            columns (string): Name of the class column.
        Returns: 
            one-hot matrix of labels of shape: [data.shape[0], 7]. 
        Raises:        
    '''    
    logger.name = 'allfeatures_extract_labels'
    if (type(columns)==str):
         indices = [i for i, elem in enumerate(data.columns) if columns in elem] # (alphabetically ordered)
    else:
        indices =  columns 

    if indices:
        labels = data[data.columns[indices]]
        data.drop(data.columns[indices], axis=1, inplace=True)    
        logger.info('...Labels extracted from Dataset...')
        return labels
    else: return None


In [21]:
def tag_chunk(tag, label, chunk, chunk_periods, tag_period, log_file, with_index, tag_index, hdf=None, tfrec=None):
    '''Extract records filtering by chunk_periods parameter, define indexes in case of with_index=True, 
        extract labels and save the results into the target file.
        Args: 
            chunk (DataFrame): Input Dataset which is modified in place.
            tag (string): 'train', 'valid' or 'test'
            chunk_periods (integer array): an array containing all periods into the chunk.
            tag_period (integer array): an array of form [init_period, end_period] for the correspond tag.
            log_file (Logger): An object of the log file.
            with_index (boolean): If true it will be saved the indexes.
            tag_index (int): an index that accumulates the size of the processed chunk. 
            hdf or tfrec (HDFStore or TFRecords): an object of the target file. Only one must be distint of None.
        Returns: 
            tag_index (int): tag_index updated.
        Raises:        
    '''    
    
    inter_periods = list(chunk_periods.intersection(set(range(tag_period[0], tag_period[1]+1))))
    log_file.write('Periods corresponding to ' + tag +' period: %s\r\n' % str(inter_periods))
    p_chunk = chunk.loc[(slice(None), inter_periods), :]
    log_file.write('Records for ' + tag +  ' Set - Number of rows: %d\r\n' % (p_chunk.shape[0]))
    print('Records for ' + tag + ' Set - Number of rows:', p_chunk.shape[0])
    if (p_chunk.shape[0] > 0):
        if (with_index==True):
            # p_chunk.index = pd.MultiIndex.from_tuples([(i, x[1], x[2],x[3]) for x,i in zip(p_chunk.index, range(tag_index, tag_index + p_chunk.shape[0]))])                                
            p_chunk.reset_index(inplace=True)
            allfeatures_drop_cols(p_chunk, ['PERIOD'])      
            p_chunk.set_index('DELINQUENCY_STATUS_NEXT', inplace=True) #1 index                                      
        else:
            p_chunk.reset_index(drop=True, inplace=True)
            
        labels = allfeatures_extract_labels(p_chunk, columns=label)
        p_chunk = p_chunk.astype(DT_FLOAT)
        labels = labels.astype(np.int8)
        if (p_chunk.shape[0] != labels.shape[0]) : 
            print('Error in shapes:', p_chunk.shape, labels.shape)
        else :
            if (hdf!=None):
                hdf.put(tag + '/features', p_chunk, append=True, index=True) #data_columns=p_chunk.columns.values), index=False
                hdf.put(tag + '/labels', labels, append=True, index=True) #data_columns=labels.columns.values)                         
                hdf.flush()                      
            elif (tfrec!=None):
                for row, lab in zip(p_chunk.values, labels.values):
                    feature = {tag + '/labels': _int64_feature(lab),
                               tag + '/features': _float_feature(row)}
                    # Create an example protocol buffer
                    example = tf.train.Example(features=tf.train.Features(feature=feature))
                    tfrec.write(example.SerializeToString())                            
                tfrec.flush()
            tag_index += p_chunk.shape[0]

    return tag_index

In [22]:
def allfeatures_drop_cols(data, columns):
    '''Exclude from the dataset 'data' the descriptive columns as parameters.
        Args: 
            data (DataFrame): Input Dataset which is modified in place.
        Returns: 
            None
        Raises:        
    '''
    logger.name = 'allfeatures_drop_cols'    
    data.drop(columns, axis=1, inplace=True)
    logger.info('...Columns Excluded from dataset...')
    return None

In [23]:
def oneHotDummies_column(column, categories):
    '''Convert categorical variable into dummy/indicator variables.
    
    Args: 
        column (Series): Input String Categorical Column.
    Returns: 
        DataFrame. Integer Sparse binary matrix of categorical features.
    Raises:        
    '''    
    logger.name = 'oneHotDummies_column: ' +  column.name
    cat_column = pd.Categorical(column.astype('str'), categories=categories)
    cat_column = pd.get_dummies(cat_column)   # in the same order as categories! (alphabetically ordered) 
    cat_column = cat_column.add_prefix(column.name + '_')
    if (cat_column.isnull().any().any()):
        null_cols = cat_column.columns[cat_column.isnull().any()]
        print(cat_column[null_cols].isnull().sum())
        print(cat_column[cat_column.isnull().any(axis=1)][null_cols].head(50))
    return cat_column

In [24]:
def imputing_nan_values(nan_dict, distribution):        
    '''Replace nan values with a value according the nan_dict dictionary and distribution of this feature.
        Args: 
            nan_dict (Dictionary): the key values are the name of features, the values could be a literal or 
            values belonging to the distribution.
            distribution (DataFrame): Contains the median value for numerical features.
        Returns: 
            new_dict (Dictionary): contains the values updated.
        Raises:        
    '''    
    new_dict = {}
    for k,v in nan_dict.items():
        if v=='median':
            new_dict[k] = float(distribution[k+'_MEDIAN'])    
        elif v=='mean':
            new_dict[k] = float(distribution[k+'_MEAN'])                
        else:
            new_dict[k] = v
            
    return new_dict

In [25]:
def drop_invalid_delinquency_status(data, gflag, log_file):   
    '''Delete all subsecuent records of a loan when the feature delinquency_status_next 
       contains any of the following invalid status: S,T,X or Z.
        Args: 
            data (DataFrame): Input Dataset which is modified in place.
            gflag (int): Loan_id of the last loan in previous data, in case this contains some invalid status, 
            to delete all records inside the current data.
            log_file (Logger): An object of the log file.
        Returns: 
            gflag (int): Loan_id of the last loan in current data, in case this contains some invalid status.
        Raises:        
    '''        
    logger.name = 'drop_invalid_delinquency_status'
    delinq_ids =  data[data['MBA_DELINQUENCY_STATUS'].isin(['0', 'R', 'S', 'T', 'X', 'Z'])]['LOAN_ID']
    groups = data[data['LOAN_ID'].isin(delinq_ids)][['LOAN_ID', 'PERIOD', 'MBA_DELINQUENCY_STATUS', 'DELINQUENCY_STATUS_NEXT']].groupby('LOAN_ID') 
    groups_list = list(groups)
    
    iuw= pd.Index([])
    
    if gflag != '': 
        try:
            iuw= iuw.union(groups.get_group(gflag).index[0:])
        except  Exception  as e:
            print(str(e))
                
    if data.iloc[-1]['LOAN_ID'] in groups.groups.keys():
        gflag = data.iloc[-1]['LOAN_ID']
    else:
        gflag = ''
                
    for k, group in groups_list: 
        li= group.index[(group['MBA_DELINQUENCY_STATUS'] =='S') | (group['MBA_DELINQUENCY_STATUS'] =='T') 
                         | (group['MBA_DELINQUENCY_STATUS'] =='X') | (group['MBA_DELINQUENCY_STATUS'] =='Z')].tolist()
        if li: iuw= iuw.union(group.index[group.index.get_loc(li[0]):])
        # In case of REO or Paid-Off, we need to exclude since the next record:
        df_delinq_01 = group[(group['MBA_DELINQUENCY_STATUS'] =='0') | (group['MBA_DELINQUENCY_STATUS'] =='R')]
        if df_delinq_01.shape[0]>0: 
            track_i = df_delinq_01.index[0]
            iuw= iuw.union(group.index[group.index.get_loc(track_i)+1:])
        
    if iuw!=[]:
        log_file.write('drop_invalid_delinquency_status - Total rows: %d\r\n' % len(iuw)) # (log_df.shape[0])
        data.drop(iuw, inplace=True) 
        logger.info('invalid_delinquency_status dropped')             
    
    return gflag

In [26]:
def zscore(x,mean,stdd):
    return (x - mean) / stdd

def zscore_apply(dist_file, data):            
    stddv_0 = []
    nnorm_cols = []
    for col_name in data.columns.values:                                
        mean = pd.Series(dist_file.iloc[0, np.where(pd.DataFrame(dist_file.columns.values)[0].str.contains(col_name+'_MEAN'))[0]], dtype='float32')    
        stddev = dist_file.iloc[0, np.where(pd.DataFrame(dist_file.columns.values)[0].str.contains(col_name+'_STDD'))[0]]    
        if not mean.empty and not stddev.empty:  
            mean = np.float32(mean.values[0])
            stddev = np.float32(stddev.values[0])            
            if stddev == 0: 
                stddv_0.append(col_name)        
            else:        
                data[col_name] = data[col_name].apply(lambda x: zscore(x, mean, stddev))                        
        else: 
            nnorm_cols.append(col_name)
    print('STANDARD DEV zero: ', stddv_0)        
    return data, nnorm_cols

In [27]:
def prepro_chunk(file_name, file_path, chunksize, label, log_file, nan_cols, categorical_cols, descriptive_cols, time_cols,
                 dist_file, with_index, refNorm, train_period, valid_period, test_period, robust_cols, minmax_cols=None, hdf=None, tfrec=None):
    gflag = ''    
    i = 1                  
    train_index = 0
    valid_index = 0
    test_index = 0
    for chunk in pd.read_csv(file_path, chunksize = chunksize, sep=',', low_memory=False):    
        print('chunk: ', i, ' chunk size: ', chunk.shape[0])
        log_file.write('chunk: %d, chunk size: %d \n' % (i, chunk.shape[0]))
        chunk.columns = chunk.columns.str.upper()                            
        
        log_df = chunk[chunk[label].isnull()]
        log_file.write('Dropping Rows with Null Labels - Number of rows: %d\r\n' % (log_df.shape[0]))
        chunk.drop(chunk.index[chunk[label].isnull()], axis=0, inplace=True)
        
        log_df = chunk[chunk['INVALID_TRANSITIONS']==1]
        log_file.write('Dropping Rows with Invalid Transitions - Number of rows: %d\r\n' % (log_df.shape[0]))                                
        chunk.drop(chunk.index[chunk['INVALID_TRANSITIONS']==1], axis=0, inplace=True)        
        
        gflag = drop_invalid_delinquency_status(chunk, gflag, log_file)               
                    
        null_columns=chunk.columns[chunk.isnull().any()]
        log_df = chunk[chunk.isnull().any(axis=1)][null_columns]
        log_file.write('Filling NULL values - (rows, cols) : %d, %d\r\n' % (log_df.shape[0], log_df.shape[1]))                    
        log_df = chunk[null_columns].isnull().sum().to_frame().reset_index()
        log_df.to_csv(log_file, index=False, mode='a')                                    
        nan_cols = imputing_nan_values(nan_cols, dist_file)            
        chunk.fillna(value=nan_cols, inplace=True)   
        
        chunk.drop_duplicates(inplace=True) # Follow this instruction!!                        
        logger.info('dropping invalid transitions and delinquency status, fill nan values, drop duplicates')                  
        log_file.write('Drop duplicates - new size : %d\r\n' % (chunk.shape[0]))
                               
        chunk.reset_index(drop=True, inplace=True)  #don't remove this line! otherwise NaN values appears.
        chunk['ORIGINATION_YEAR'][chunk['ORIGINATION_YEAR']<1995] = "B1995"
        for k,v in categorical_cols.items():
            # if (chunk[k].dtype=='O'):                
            chunk[k] = chunk[k].astype('str')
            chunk[k] = chunk[k].str.strip()
            chunk[k].replace(['\.0$'], [''], regex=True,  inplace=True)
            new_cols = oneHotDummies_column(chunk[k], v)
            if (chunk[k].value_counts().sum()!=new_cols.sum().sum()):
                print('Error at categorization, different sizes', k)
                print(chunk[k].value_counts(), new_cols.sum())                
                log_file.write('Error at categorization, different sizes %s\r\n' % str(k))
                chunk[new_cols.columns] = new_cols
            else:
                chunk[new_cols.columns] = new_cols
                log_file.write('New columns added: %s\r\n' % str(new_cols.columns.values))
            
                    
        allfeatures_drop_cols(chunk, descriptive_cols)                    
        #np.savetxt(log_file, descriptive_cols, header='descriptive_cols dropped:', newline=" ")
        log_file.write('descriptive_cols dropped: %s\r\n' % str(descriptive_cols))
        allfeatures_drop_cols(chunk, time_cols)
        #np.savetxt(log_file, time_cols, header='time_cols dropped:', newline=" ")
        log_file.write('time_cols dropped: %s\r\n' % str(time_cols))
        cat_list = list(categorical_cols.keys())
        cat_list.remove('DELINQUENCY_STATUS_NEXT')
        #np.savetxt(log_file, cat_list, header='categorical_cols dropped:', newline=" ")
        log_file.write('categorical_cols dropped: %s\r\n' % str(cat_list))
        allfeatures_drop_cols(chunk, cat_list)

        chunk.reset_index(drop=True, inplace=True)  
        chunk.set_index(['DELINQUENCY_STATUS_NEXT', 'PERIOD'], append=False, inplace=True) #2 indexes
        # np.savetxt(log_file, str(chunk.index.names), header='Indexes created:', newline=" ")
        log_file.write('Indexes created: %s\r\n' % str(chunk.index.names))
         
        
        
        if chunk.isnull().any().any(): 
            # from IPython.core.debugger import Tracer; Tracer()()
            raise ValueError('There are null values...File: ' + file_name)   
                
        
        if (refNorm==True):            
            chunk[robust_cols], nnorm_cols =  zscore_apply(dist_file, chunk[robust_cols]) #robust_normalizer.transform(chunk[robust_cols])            
            log_file.write('Columns not normalized: %s\r\n' % str(nnorm_cols))            
            log_file.write('Columns normalized: %s\r\n' % str(set(robust_cols)-set(nnorm_cols)))
            
        
        if chunk.isnull().any().any(): raise ValueError('There are null values...File: ' + file_name)       
        
        chunk_periods = set(list(chunk.index.get_level_values('PERIOD')))
        #print(tfrec)
        if (tfrec!=None):
            train_index = tag_chunk('train', label, chunk, chunk_periods, train_period, log_file, with_index, train_index, tfrec=tfrec[0])
            valid_index = tag_chunk('valid', label, chunk, chunk_periods, valid_period, log_file, with_index, valid_index, tfrec=tfrec[1])
            test_index = tag_chunk('test', label, chunk, chunk_periods, test_period, log_file, with_index, test_index, tfrec=tfrec[2])
            sys.stdout.flush()
        elif (hdf!=None):
            train_index = tag_chunk('train', label, chunk, chunk_periods, train_period, log_file, with_index, train_index, hdf=hdf[0])
            valid_index = tag_chunk('valid', label, chunk, chunk_periods, valid_period, log_file, with_index, valid_index, hdf=hdf[1])
            test_index = tag_chunk('test', label, chunk, chunk_periods, test_period, log_file, with_index, test_index, hdf=hdf[2])                
        
        inter_periods = list(chunk_periods.intersection(set(range(test_period[1]+1,355))))    
        log_file.write('Periods greater than test_period: %s\r\n' % str(inter_periods))
        p_chunk = chunk.loc[(slice(None), inter_periods), :]
        log_file.write('Records greater than test_period - Number of rows: %d\r\n' % (p_chunk.shape[0]))
        
        del chunk        
        i +=  1   
    
    return train_index, valid_index, test_index

In [28]:
def custom_robust_normalizer(ncols, dist_file, normalizer_type='robust_scaler_sk', center_value='median'):            
    norm_cols = []
    scales = []
    centers = []
    scales_0 =[]
    for i, x in enumerate (ncols):                        
        x_frame = dist_file.iloc[:, np.where(pd.DataFrame(dist_file.columns.values)[0].str.contains(x+'_Q'))[0]]    
        if not x_frame.empty and (x_frame.shape[1]>1):       
            iqr = float(pd.to_numeric(x_frame[x+'_Q3'], errors='coerce').subtract(pd.to_numeric(x_frame[x+'_Q1'], errors='coerce')))
            if iqr == 0: scales_0.append(x)
            if iqr!=0: 
                norm_cols.append(x)                
                scales.append(iqr)                    
                if center_value == 'median':
                    centers.append( float(x_frame[x+'_MEDIAN']) )   
                else:
                    centers.append( float(x_frame[x+'_Q1']) )                                       
    if (normalizer_type == 'robust_scaler_sk'):    
        normalizer = RobustScaler()
        normalizer.scale_ = scales
        normalizer.center_ = centers        
    elif (normalizer_type == 'percentile_scaler'):    
        normalizer = Normalizer.Normalizer(scales, centers)     
    else: normalizer=None                  
    
    print(scales_0)
    
    return norm_cols, normalizer

In [29]:
def custom_minmax_normalizer(ncols, scales, dist_file):    
    norm_cols = []
    minmax_scales = []
    centers = []
    for i, x in enumerate (ncols):  
        x_min = dist_file.iloc[0, np.where(pd.DataFrame(dist_file.columns.values)[0].str.contains(x+'_MIN'))[0]]
        x_max = dist_file.iloc[0, np.where(pd.DataFrame(dist_file.columns.values)[0].str.contains(x+'_MAX'))[0]]
        if not(x_min.empty) and not(x_max.empty):            
            x_min = np.float32(x_min.values[0])
            x_max = np.float32(x_max.values[0])
            minmax_scales.append(x_max - x_min)                            
            centers.append(x_min)
            norm_cols.append(x)
            # to_delete.append(i)
        
    normalizer = Normalizer.Normalizer(minmax_scales, centers)         
    
    return norm_cols, normalizer #, to_delete

In [30]:
def allfeatures_preprocessing(RAW_DIR, PRO_DIR, raw_dir, train_period, valid_period, test_period, dividing='percentage', 
                              chunksize=500000, refNorm=True, with_index=True, output_hdf=True, label='DELINQUENCY_STATUS_NEXT'):            

    descriptive_cols = [
    'LOAN_ID',
    'ASOFMONTH',        
    'PERIOD_NEXT',
    'MOD_PER_FROM',
    'MOD_PER_TO',
    'PROPERTY_ZIP',
    'INVALID_TRANSITIONS'
    ]

    numeric_cols = ['MBA_DAYS_DELINQUENT', 'MBA_DAYS_DELINQUENT_NAN',
       'CURRENT_INTEREST_RATE', 'CURRENT_INTEREST_RATE_NAN', 'LOANAGE', 'LOANAGE_NAN',
       'CURRENT_BALANCE', 'CURRENT_BALANCE_NAN', 'SCHEDULED_PRINCIPAL',
       'SCHEDULED_PRINCIPAL_NAN', 'SCHEDULED_MONTHLY_PANDI',
       'SCHEDULED_MONTHLY_PANDI_NAN', 
       'LLMA2_CURRENT_INTEREST_SPREAD', 'LLMA2_CURRENT_INTEREST_SPREAD_NAN',  
       'LLMA2_C_IN_LAST_12_MONTHS',
       'LLMA2_30_IN_LAST_12_MONTHS', 'LLMA2_60_IN_LAST_12_MONTHS',
       'LLMA2_90_IN_LAST_12_MONTHS', 'LLMA2_FC_IN_LAST_12_MONTHS',
       'LLMA2_REO_IN_LAST_12_MONTHS', 'LLMA2_0_IN_LAST_12_MONTHS',
       'LLMA2_HIST_LAST_12_MONTHS_MIS', 
       'NUM_MODIF', 'NUM_MODIF_NAN', 'P_RATE_TO_MOD', 'P_RATE_TO_MOD_NAN', 'MOD_RATE',
       'MOD_RATE_NAN', 'DIF_RATE', 'DIF_RATE_NAN', 'P_MONTHLY_PAY',
       'P_MONTHLY_PAY_NAN', 'MOD_MONTHLY_PAY', 'MOD_MONTHLY_PAY_NAN',
       'DIF_MONTHLY_PAY', 'DIF_MONTHLY_PAY_NAN', 'CAPITALIZATION_AMT',
       'CAPITALIZATION_AMT_NAN', 'MORTGAGE_RATE', 'MORTGAGE_RATE_NAN',
       'FICO_SCORE_ORIGINATION', 'INITIAL_INTEREST_RATE', 'ORIGINAL_LTV',
       'ORIGINAL_BALANCE', 'BACKEND_RATIO', 'BACKEND_RATIO_NAN',
       'ORIGINAL_TERM', 'ORIGINAL_TERM_NAN', 'SALE_PRICE', 'SALE_PRICE_NAN', 	   
       'PREPAY_PENALTY_TERM', 'PREPAY_PENALTY_TERM_NAN', 
        'NUMBER_OF_UNITS', 'NUMBER_OF_UNITS_NAN', 'MARGIN',
       'MARGIN_NAN', 'PERIODIC_RATE_CAP', 'PERIODIC_RATE_CAP_NAN',
       'PERIODIC_RATE_FLOOR', 'PERIODIC_RATE_FLOOR_NAN', 'LIFETIME_RATE_CAP',
       'LIFETIME_RATE_CAP_NAN', 'LIFETIME_RATE_FLOOR',
       'LIFETIME_RATE_FLOOR_NAN', 'RATE_RESET_FREQUENCY',
       'RATE_RESET_FREQUENCY_NAN', 'PAY_RESET_FREQUENCY',
       'PAY_RESET_FREQUENCY_NAN', 'FIRST_RATE_RESET_PERIOD',
       'FIRST_RATE_RESET_PERIOD_NAN', 	   
        'LLMA2_PRIME',
       'LLMA2_SUBPRIME', 'LLMA2_APPVAL_LT_SALEPRICE', 'LLMA2_ORIG_RATE_SPREAD',
       'LLMA2_ORIG_RATE_SPREAD_NAN', 'AGI', 'AGI_NAN', 'UR', 'UR_NAN', 'LLMA2_ORIG_RATE_ORIG_MR_SPREAD', 
       'LLMA2_ORIG_RATE_ORIG_MR_SPREAD_NAN', 'COUNT_INT_RATE_LESS', 'NUM_PRIME_ZIP', 'NUM_PRIME_ZIP_NAN'
       ]

    '''
    nan_cols = {'MBA_DAYS_DELINQUENT': 'median', 'CURRENT_INTEREST_RATE': 'median', 'LOANAGE': 'median',
                'CURRENT_BALANCE' : 'median', 'SCHEDULED_PRINCIPAL': 'median', 'SCHEDULED_MONTHLY_PANDI': 'median',       
                'LLMA2_CURRENT_INTEREST_SPREAD': 'median', 'NUM_MODIF': 0, 'P_RATE_TO_MOD': 0, 'MOD_RATE': 0,
                'DIF_RATE': 0, 'P_MONTHLY_PAY': 0, 'MOD_MONTHLY_PAY': 0, 'DIF_MONTHLY_PAY': 0, 'CAPITALIZATION_AMT': 0,
                'MORTGAGE_RATE': 'median', 'FICO_SCORE_ORIGINATION': 'median', 'INITIAL_INTEREST_RATE': 'median', 'ORIGINAL_LTV': 'median',
                'ORIGINAL_BALANCE': 'median', 'BACKEND_RATIO': 'median', 'ORIGINAL_TERM': 'median', 'SALE_PRICE': 'median', 'PREPAY_PENALTY_TERM': 'median',
                'NUMBER_OF_UNITS': 'median', 'MARGIN': 'median', 'PERIODIC_RATE_CAP': 'median', 'PERIODIC_RATE_FLOOR': 'median', 'LIFETIME_RATE_CAP': 'median',
                'LIFETIME_RATE_FLOOR': 'median', 'RATE_RESET_FREQUENCY': 'median', 'PAY_RESET_FREQUENCY': 'median',
                'FIRST_RATE_RESET_PERIOD': 'median', 'LLMA2_ORIG_RATE_SPREAD': 'median', 'AGI': 'median', 'UR': 'median',
                'LLMA2_C_IN_LAST_12_MONTHS': 'median', 'LLMA2_30_IN_LAST_12_MONTHS': 'median', 'LLMA2_60_IN_LAST_12_MONTHS': 'median',
                'LLMA2_90_IN_LAST_12_MONTHS': 'median', 'LLMA2_FC_IN_LAST_12_MONTHS': 'median',
                'LLMA2_REO_IN_LAST_12_MONTHS': 'median', 'LLMA2_0_IN_LAST_12_MONTHS': 'median', 
                'LLMA2_ORIG_RATE_ORIG_MR_SPREAD':0, 'COUNT_INT_RATE_LESS' :'median', 'NUM_PRIME_ZIP':'median'
                }
    '''
    nan_cols = {'MBA_DAYS_DELINQUENT': 'mean', 'CURRENT_INTEREST_RATE': 'mean', 'LOANAGE': 'mean',
                'CURRENT_BALANCE' : 'mean', 'SCHEDULED_PRINCIPAL': 'mean', 'SCHEDULED_MONTHLY_PANDI': 'mean',       
                'LLMA2_CURRENT_INTEREST_SPREAD': 'mean', 'NUM_MODIF': 0, 'P_RATE_TO_MOD': 0, 'MOD_RATE': 0,
                'DIF_RATE': 0, 'P_MONTHLY_PAY': 0, 'MOD_MONTHLY_PAY': 0, 'DIF_MONTHLY_PAY': 0, 'CAPITALIZATION_AMT': 0,
                'MORTGAGE_RATE': 'mean', 'FICO_SCORE_ORIGINATION': 'mean', 'INITIAL_INTEREST_RATE': 'mean', 'ORIGINAL_LTV': 'mean',
                'ORIGINAL_BALANCE': 'mean', 'BACKEND_RATIO': 'mean', 'ORIGINAL_TERM': 'mean', 'SALE_PRICE': 'mean', 'PREPAY_PENALTY_TERM': 'mean',
                'NUMBER_OF_UNITS': 'mean', 'MARGIN': 'mean', 'PERIODIC_RATE_CAP': 'mean', 'PERIODIC_RATE_FLOOR': 'mean', 'LIFETIME_RATE_CAP': 'mean',
                'LIFETIME_RATE_FLOOR': 'mean', 'RATE_RESET_FREQUENCY': 'mean', 'PAY_RESET_FREQUENCY': 'mean',
                'FIRST_RATE_RESET_PERIOD': 'mean', 'LLMA2_ORIG_RATE_SPREAD': 'mean', 'AGI': 'mean', 'UR': 'mean',
                'LLMA2_C_IN_LAST_12_MONTHS': 'mean', 'LLMA2_30_IN_LAST_12_MONTHS': 'mean', 'LLMA2_60_IN_LAST_12_MONTHS': 'mean',
                'LLMA2_90_IN_LAST_12_MONTHS': 'mean', 'LLMA2_FC_IN_LAST_12_MONTHS': 'mean',
                'LLMA2_REO_IN_LAST_12_MONTHS': 'mean', 'LLMA2_0_IN_LAST_12_MONTHS': 'mean', 
                'LLMA2_ORIG_RATE_ORIG_MR_SPREAD':0, 'COUNT_INT_RATE_LESS' :'mean', 'NUM_PRIME_ZIP':'mean'
                }
    
    categorical_cols = {'MBA_DELINQUENCY_STATUS':  ['0','3','6','9','C','F','R'], 'DELINQUENCY_STATUS_NEXT': ['0','3','6','9','C','F','R'],  #,'S','T','X'
                           'BUYDOWN_FLAG': ['N','U','Y'], 'NEGATIVE_AMORTIZATION_FLAG': ['N','U','Y'], 'PREPAY_PENALTY_FLAG': ['N','U','Y'],
                           'OCCUPANCY_TYPE': ['1','2','3','U'], 'PRODUCT_TYPE': ['10','20','30','40','50','51','52','53','54','5A','5Z',
                                            '60','61','62','63','6Z','70','80','81','82','83','84','8Z','U'], 
                           'PROPERTY_TYPE': ['1','2','3','4','5','6','7','8','9','M','U','Z'], 'LOAN_PURPOSE_CATEGORY': ['P','R','U'], 
                           'DOCUMENTATION_TYPE': ['1','2','3','U'], 'CHANNEL': ['1','2','3','4','5','6','7','8','9','A','B','C','D','U'], 
                           'LOAN_TYPE': ['1','2','3','4','5','6','U'], 'IO_FLAG': ['N','U','Y'], 
                           'CONVERTIBLE_FLAG': ['N','U','Y'], 'POOL_INSURANCE_FLAG': ['N','U','Y'], 'STATE': ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO',
                                               'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 
                                               'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 
                                               'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 
                                               'WA', 'WI', 'WV', 'WY'], 
                           'CURRENT_INVESTOR_CODE': ['240', '250', '253', 'U'], 'ORIGINATION_YEAR': ['B1995','1995','1996','1997','1998','1999','2000','2001','2002','2003',
                                                    '2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']}

    time_cols = ['YEAR', 'MONTH'] #, 'PERIOD'] #no nan values        

    total_cols = numeric_cols.copy() 
    total_cols.extend(descriptive_cols)
    total_cols.extend(categorical_cols.keys())
    total_cols.extend(time_cols)
    print('total_cols size: ', len(total_cols)) #110 !=112?? set(chunk_cols) - set(total_cols): {'LOAN_ID', 'PERIOD'}
    
    pd.set_option('io.hdf.default_format','table')

    dist_file = pd.read_csv(os.path.join(RAW_DIR, "percentile features3-mean.csv"), sep=';', low_memory=False)
    dist_file.columns = dist_file.columns.str.upper()

    ncols = [x for x in numeric_cols if x.find('NAN')<0]
    print(ncols)

    #sum = 0
    #for elem in categorical_cols.values():
    #    sum += len(elem)
    #print('total categorical values: ', sum) #181

    for file_path in glob.glob(os.path.join(RAW_DIR, raw_dir,"*.txt")):  
        file_name = os.path.basename(file_path)
        if with_index==True:
            target_path = os.path.join(PRO_DIR, raw_dir,file_name[:-4])        
        else:
            target_path = os.path.join(PRO_DIR, raw_dir,file_name[:-4]+'_non_index')
        log_file=open(target_path+'-log.txt', 'w+', 1)        
        print('Preprocessing File: ' + file_path)
        log_file.write('Preprocessing File:  %s\r\n' % file_path)
        startTime = datetime.now()      
        
        if (output_hdf == True):
            #with  pd.HDFStore(target_path +'-pp.h5', complib='lzo', complevel=9) as hdf: #complib='lzo', complevel=9
            train_writer = pd.HDFStore(target_path +'-train_.h5', complib='lzo', complevel=9) 
            valid_writer = pd.HDFStore(target_path +'-valid_.h5', complib='lzo', complevel=9)
            test_writer = pd.HDFStore(target_path +'-test_.h5', complib='lzo', complevel=9) 

            print('generating: ', target_path +'-pp.h5')
            train_index, valid_index, test_index = prepro_chunk(file_name, file_path, chunksize, label, log_file, 
                                                                nan_cols, categorical_cols, descriptive_cols, time_cols,
                                                                dist_file, with_index, 
                                                                refNorm, train_period, valid_period, test_period, ncols,                                                                
                                                                hdf=[train_writer, valid_writer, test_writer], tfrec=None)            


            if train_writer.get_storer('train/features').nrows != train_writer.get_storer('train/labels').nrows:
                    raise ValueError('Train-DataSet: Sizes should match!')  
            if valid_writer.get_storer('valid/features').nrows != valid_writer.get_storer('valid/labels').nrows:
                    raise ValueError('Valid-DataSet: Sizes should match!')  
            if test_writer.get_storer('test/features').nrows != test_writer.get_storer('test/labels').nrows:
                    raise ValueError('Test-DataSet: Sizes should match!')  

            print('train/features size: ', train_writer.get_storer('train/features').nrows)
            print('valid/features size: ', valid_writer.get_storer('valid/features').nrows)
            print('test/features size: ', test_writer.get_storer('test/features').nrows)

            log_file.write('***SUMMARY***\n')
            log_file.write('train/features size: %d\r\n' %(train_writer.get_storer('train/features').nrows))
            log_file.write('valid/features size: %d\r\n' %(valid_writer.get_storer('valid/features').nrows))
            log_file.write('test/features size: %d\r\n' %(test_writer.get_storer('test/features').nrows))

            logger.info('training, validation and testing set into .h5 file')        
        else:        
            train_writer = tf.python_io.TFRecordWriter(target_path +'-train_.tfrecords')
            valid_writer = tf.python_io.TFRecordWriter(target_path +'-valid_.tfrecords')
            test_writer = tf.python_io.TFRecordWriter(target_path +'-test_.tfrecords')
            train_index, valid_index, test_index = prepro_chunk(file_name, file_path, chunksize, label, log_file, 
                                                                nan_cols, categorical_cols, descriptive_cols, time_cols,
                                                                dist_file, with_index, 
                                                                refNorm, train_period, valid_period, test_period, ncols,
                                                                hdf=None, tfrec=[train_writer, valid_writer, test_writer]) 
        print(train_index, valid_index, test_index)
        train_writer.close()
        valid_writer.close()
        test_writer.close()        
        
        #def allfeatures_prepro_file(RAW_DIR, file_path, raw_dir, file_name, target_path, train_period, valid_period, test_period, log_file, dividing='percentage', chunksize=500000, 
        #                    refNorm=True, , with_index=True, output_hdf=True):

        #allfeatures_prepro_file(RAW_DIR, file_path, raw_dir, file_name, target_path, train_num, valid_num, test_num, log_file, dividing=dividing, chunksize=chunksize, 
        #                        refNorm=refNorm, with_index=with_index, output_hdf=output_hdf)          
        
        startTime = datetime.now() - startTime
        print('Preprocessing Time per file: ', startTime)     
        log_file.write('Preprocessing Time per file:  %s\r\n' % str(startTime))
        log_file.close()


In [31]:
startTime = datetime.now()

if not os.path.exists(os.path.join(PRO_DIR, FLAGS.prepro_dir)): #os.path.exists
    os.makedirs(os.path.join(PRO_DIR, FLAGS.prepro_dir))

allfeatures_preprocessing(RAW_DIR, PRO_DIR, FLAGS.prepro_dir, FLAGS.train_period, FLAGS.valid_period, FLAGS.test_period, dividing='percentage', 
                          chunksize=FLAGS.prepro_chunksize, refNorm=FLAGS.ref_norm, with_index=FLAGS.prepro_with_index, output_hdf=True)        
print('Preprocessing - Time: ', datetime.now() - startTime)

total_cols size:  111
['MBA_DAYS_DELINQUENT', 'CURRENT_INTEREST_RATE', 'LOANAGE', 'CURRENT_BALANCE', 'SCHEDULED_PRINCIPAL', 'SCHEDULED_MONTHLY_PANDI', 'LLMA2_CURRENT_INTEREST_SPREAD', 'LLMA2_C_IN_LAST_12_MONTHS', 'LLMA2_30_IN_LAST_12_MONTHS', 'LLMA2_60_IN_LAST_12_MONTHS', 'LLMA2_90_IN_LAST_12_MONTHS', 'LLMA2_FC_IN_LAST_12_MONTHS', 'LLMA2_REO_IN_LAST_12_MONTHS', 'LLMA2_0_IN_LAST_12_MONTHS', 'LLMA2_HIST_LAST_12_MONTHS_MIS', 'NUM_MODIF', 'P_RATE_TO_MOD', 'MOD_RATE', 'DIF_RATE', 'P_MONTHLY_PAY', 'MOD_MONTHLY_PAY', 'DIF_MONTHLY_PAY', 'CAPITALIZATION_AMT', 'MORTGAGE_RATE', 'FICO_SCORE_ORIGINATION', 'INITIAL_INTEREST_RATE', 'ORIGINAL_LTV', 'ORIGINAL_BALANCE', 'BACKEND_RATIO', 'ORIGINAL_TERM', 'SALE_PRICE', 'PREPAY_PENALTY_TERM', 'NUMBER_OF_UNITS', 'MARGIN', 'PERIODIC_RATE_CAP', 'PERIODIC_RATE_FLOOR', 'LIFETIME_RATE_CAP', 'LIFETIME_RATE_FLOOR', 'RATE_RESET_FREQUENCY', 'PAY_RESET_FREQUENCY', 'FIRST_RATE_RESET_PERIOD', 'LLMA2_PRIME', 'LLMA2_SUBPRIME', 'LLMA2_APPVAL_LT_SALEPRICE', 'LLMA2_ORIG_RAT

2018-11-20 18:12:04,916 - drop_invalid_delinquency_status - INFO - invalid_delinquency_status dropped
2018-11-20 18:12:09,601 - drop_invalid_delinquency_status - INFO - dropping invalid transitions and delinquency status, fill nan values, drop duplicates
C:\Users\sandr\AppData\Local\conda\conda\envs\tensorflowenvironment\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
2018-11-20 18:12:22,161 - allfeatures_drop_cols - INFO - ...Columns Excluded from dataset...
2018-11-20 18:12:22,541 - allfeatures_drop_cols - INFO - ...Columns Excluded from dataset...
2018-11-20 18:12:22,884 - allfeatures_drop_cols - INFO - ...Columns Excluded from dataset...
C:\Users\sandr\AppData\Local\conda\conda\envs\tensorflowenvironment\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A v

STANDARD DEV zero:  []
Records for train Set - Number of rows: 400455


C:\Users\sandr\AppData\Local\conda\conda\envs\tensorflowenvironment\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
2018-11-20 18:13:26,375 - allfeatures_drop_cols - INFO - ...Columns Excluded from dataset...
C:\Users\sandr\AppData\Local\conda\conda\envs\tensorflowenvironment\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
2018-11-20 18:13:26,950 - allfeatures_extract_labels - INFO - ...Labels extracted from Dataset...
2018-11-20 18:13:30,561 - allfeatures_drop_cols - INFO - ...Columns Excluded from dataset...
2018-11-20

Records for valid Set - Number of rows: 9044


...

Records for valid Set - Number of rows: 8916


2018-11-20 20:05:07,639 - allfeatures_drop_cols - INFO - ...Columns Excluded from dataset...
2018-11-20 20:05:07,686 - allfeatures_extract_labels - INFO - ...Labels extracted from Dataset...


Records for test Set - Number of rows: 16525


2018-11-20 20:05:08,006 - allfeatures_extract_labels - INFO - training, validation and testing set into .h5 file


train/features size:  26769588
valid/features size:  901868
test/features size:  1915122
26769588 901868 1915122
Preprocessing Time per file:  1:53:22.908643
Preprocessing - Time:  1:53:23.031321
